In [13]:
import requests
import urllib
import pandas as pd

In [14]:
#import and read the csv
dfSat = pd.read_csv("viirs-snpp_2021_United_States.csv") #change this to the FIRMS csv download file name you want
dfSat

,Unnamed: 0,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type
0,0,43.476295,-71.587440,297.23,0.48,0.40,2021-01-01,624,N,VIIRS,n,1,269.23,0.68,N,0
1,1,43.246197,-72.420631,298.24,0.52,0.42,2021-01-01,624,N,VIIRS,n,1,267.55,1.06,N,0
2,2,40.890423,-77.711998,318.41,0.59,0.53,2021-01-01,624,N,VIIRS,n,1,269.67,3.09,N,2
3,3,39.589787,-75.633263,295.77,0.44,0.46,2021-01-01,624,N,VIIRS,n,1,276.95,1.78,N,2
4,4,27.574802,-81.772072,298.91,0.37,0.58,2021-01-01,624,N,VIIRS,n,1,284.31,0.51,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661053,661053,29.796158,-90.769928,333.09,0.40,0.37,2021-12-31,1900,N,VIIRS,n,1,299.65,11.39,D,0
661054,661054,29.784075,-91.766235,326.32,0.42,0.38,2021-12-31,1900,N,VIIRS,n,1,295.23,3.28,D,0
661055,661055,40.144726,-101.530975,342.62,0.51,0.50,2021-12-31,1906,N,VIIRS,n,1,277.59,7.29,D,0
661056,661056,38.554855,-122.550667,334.54,0.52,0.41,2021-12-31,2042,N,VIIRS,n,1,281.30,8.67,D,0


In [3]:
# USGS Elevation Point Query Service
url = r'https://nationalmap.gov/epqs/pqs.php?'

# coordinates with known elevation 
lat = [43.476295, 43.246197, 40.890423, 39.589787]
log = [-71.587440, -72.420631, -77.711998, -75.633263]


# create data frame
df = pd.DataFrame({
    'lat': lat,
    'lon': log
})

def elevation_function(df, lat_column, lon_column):
    """Query service using lat, lon. add the elevation values as a new column."""
    elevations = []
    for lat, lon in zip(df[lat_column], df[lon_column]):

        # define rest query params
        params = {
            'output': 'json',
            'x': lon,
            'y': lat,
            'units': 'Meters'
        }

        # format query string and return query value
        result = requests.get((url + urllib.parse.urlencode(params)))
        elevations.append(result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])

    df['elev_meters'] = elevations

elevation_function(df, 'lat', 'lon')
df.head()

,lat,lon,elev_meters
0,43.476295,-71.587440,208.31
1,43.246197,-72.420631,143.97
2,40.890423,-77.711998,322.79
3,39.589787,-75.633263,18.82


In [4]:
# USGS Elevation Point Query Service
url = r'https://nationalmap.gov/epqs/pqs.php?'

# coordinates with known elevation 
lat = dfSat['latitude']
log = dfSat['longitude']

# create data frame
df = pd.DataFrame({
    'lat': lat,
    'lon': log
})

def elevation_function(df, lat_column, lon_column, block_size=50000):
    """Query service using lat, lon. add the elevation values as a new column."""
    elevations = []
    for lat, lon in zip(df[lat_column], df[lon_column]):

        # define rest query params
        params = {
            'output': 'json',
            'x': lon,
            'y': lat,
            'units': 'Meters'
        }

        # format query string and return query value
        result = requests.get((url + urllib.parse.urlencode(params)))
        elevations.append(result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])

    df['elev_meters'] = elevations

elevation_function(df, 'lat', 'lon')
df.head()

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [35]:
def make_remote_request(url: str, params: dict):
   """
   Makes the remote request
   Continues making attempts until it succeeds
   """

   count = 1
   while True:
       try:
           response = requests.get((url + urllib.parse.urlencode(params)))
       except (OSError, urllib3.exceptions.ProtocolError) as error:
           print('\n')
           print('*' * 20, 'Error Occured', '*' * 20)
           print(f'Number of tries: {count}')
           print(f'URL: {url}')
           print(error)
           print('\n')
           count += 1
           continue
       break

   return response


def elevation_function(x):
   #url = 'https://api.opentopodata.org/v1/eudem25m?'
   url = 'https://api.open-elevation.com/api/v1/lookup?'
   params = {'locations': f"{x[0]},{x[1]}"}
   result = make_remote_request(url, params)
   return result.json()['results'][0]['elevation']

In [36]:
dfSat['elevation'] = dfSat.apply(elevation_function, axis=1)


# Print output
dfSat

NameError: name 'urllib3' is not defined

In [24]:
def get_elevation(lat, long):
    
    #if lat is None or Long is None: return None

    query = ('https://api.open-elevation.com/api/v1/lookup'
             f'?locations={lat},{long}')
    r = requests.get(query).json()
    elevation = pd.io.json.json_normalizer(r, 'results')['elevation'].values[0]
    return elevation

dfSat['elevation'] = elevation

get_elevation(dfSat, 'lat', 'long')
dfSat.head()

NameError: name 'elevation' is not defined

if r.status_code == 200 or r.status_code == 201:
        elevation = json_normalizer(r.json(), 'results')['elevation'].values[0]
    else:
        elevation = None
    return elevation

try:
    response = reqests.get('',timeout=5)
    response.raise_for_status()
    #code here will only run if the request is successful
except requests.exceptions.HTTPError as errh:
    print(errh)
except requests.exceptions.ConnectionError as errc:
    print(errc)
except requests.exceptions.Timeout as errt:
    print(errt)
except requests.exceptions.RequestException as err:
    print(err)

# USGS Elevation Point Query Service
url = r'https://nationalmap.gov/epqs/pqs.php?'

def make_remote_request(url: str, params: dict):
   """
   Makes the remote request
   Continues making attempts until it succeeds
   """

   count = 1
   while True:
       try:
           response = requests.get((url + urllib.parse.urlencode(params)))
           response.raise_for_status()
       except requests.exceptions.HTTPError as errh:
           print(errh)
       except requests.exceptions.ConnectionError as errc:
           print(errc)
       except requests.exceptions.Timeout as errt:
           print(errt)
       except requests.exceptions.RequestException as err:
           print(err)
           count += 1
           continue
       break

   return response


def elevation_function(x):
   url = 'https://nationalmap.gov/epqs/pqs.php?'
   params = {'x': x[1],
             'y': x[0],
             'units': 'Meters',
             'output': 'json'}
   result = make_remote_request(url, params)
   return result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation']

dfSat['elevation'] = dfSat.apply(elevation_function, axis=1)


# Print output
dfSat.head()